In [1]:
import pandas as pd

# Binary or 3 label prediction?

Choose the label correctly


In [2]:
if_binary = 0

Compile the dataset now

In [3]:
df =pd.read_csv("/Users/akseljoonas/Documents/predtrade/stock_bars_final.csv")

In [4]:
# Check how many cases have the same values in 'summary' and 'description'
matching_cases_count = (df["summary"] == df["description"]).sum()

# Output the result
print(
    f"Number of cases where 'summary' and 'description' are the same: {matching_cases_count} of lenght {len(df)}"
)

Number of cases where 'summary' and 'description' are the same: 3901 of lenght 3901


In [5]:
# 1. Merge the text in 'title', 'summary', and 'description' into a new column 'text'
df["text"] = (
    df["title"].fillna("")
    + " "
    + df["summary"].fillna("")
)


In [6]:
# 2. Remove '<>:/\|?*' characters from the 'text' column
df["text"] = df["text"].str.replace(r"[<>:/\\|?*]", "", regex=True)


In [7]:
df.rename(columns={"increased_3_percent": "labels"}, inplace=True)

In [8]:
if not if_binary:

    def label_price_move(row):
        if row < -0.03:
            return 0
        elif row > 0.03:
            return 2
        else:
            return 1


    # Apply the labeling function to the price_move column
    df["labels"] = df["price_move"].apply(label_price_move)

In [9]:
df = df[["text", 'topic' ,"labels"]]

In [10]:
df.drop_duplicates(inplace=True)

In [75]:
df_relevant = df[df["labels"].isin([0, 2])]
df_irrelevant = df[df["labels"].isin([1])]
result = (
    df_relevant.groupby("topic")["labels"]
    .count()
    .reset_index(name="relevant_label_count")
)
result1 = (
    df_irrelevant.groupby("topic")["labels"]
    .count()
    .reset_index(name="irrelevant_label_count")
)
result = pd.merge(result, result1, on='topic', how="outer")

result_sorted = result.sort_values(by="relevant_label_count", ascending=False)
result_sorted

,topic,relevant_label_count,irrelevant_label_count
7,clinical_study,194.0,393.0
15,earnings_releases_and_operating_results,128.0,259.0
10,conference_calls__webcasts,111.0,381.0
43,product___services_announcement,94.0,222.0
26,health,74.0,199.0
3,calendar_of_events,67.0,314.0
18,european_regulatory_news,50.0,258.0
23,financing_agreements,48.0,55.0
42,press_releases,47.0,101.0
4,changes_in_companys_own_shares,31.0,67.0


In [76]:
min_label_count = 5

excluded_topics = [
    "calendar_of_events",
    "conference_calls__webcasts",
    "directors_and_officers",
    "company_announcements",
    "european_regulatory_news",
]


# Filter topics with relevant_label_count >= 7
filtered_topics = result_sorted[
    result_sorted["relevant_label_count"] >= min_label_count
]["topic"].tolist()

# Remove excluded topics from the filtered topics
filtered_topics = [topic for topic in filtered_topics if topic not in excluded_topics]

# Now filter the original DataFrame based on the remaining filtered topics
df = df[df["topic"].isin(filtered_topics)]

# df.drop('topic',inplace=True, axis=1)
# Optionally, reset the index for cleanliness
df.reset_index(drop=True, inplace=True)

In [11]:
df['labels'].value_counts()

labels
1    2807
2     612
0     461
Name: count, dtype: int64

In [12]:
df.drop('topic',inplace=True, axis=1)

In [13]:
# 4. Save the modified DataFrame to a new CSV file
df.to_csv(
    "./datasets_big/dataset_finetuning_3_labels.csv",
    index=False,
)